In [95]:
import polars as pl
from pathlib import Path


# load pareto sets
problem_name = "mystery"
file_ext = "parquet"
n_gen_front = 10_000
psize_front = 1_000
# optimal_value = -268.78792  # branin
optimal_value = -1.174261  # mystery

fronts_path = Path("results/fronts/")
front_file_name = Path(fronts_path / f"{problem_name}_gen{n_gen_front}_psize{psize_front}.{file_ext}")

df_front = pl.read_parquet(front_file_name)
df_front.filter(pl.col("c_1") <= 0).sort("f_1")

x_1,x_2,f_1,c_1,f_1_min,c_1_min,generation
f64,f64,f64,f64,f64,f64,i32
2.745316,2.352611,-1.174261,-0.000006,-1.174261,-0.000006,3384
2.74457,2.351859,-1.174253,-0.000012,-1.174253,-0.000012,1032
2.745477,2.352731,-1.174206,-0.000047,-1.174206,-0.000047,7971
2.74558,2.352823,-1.174187,-0.000059,-1.174187,-0.000059,6853
2.744324,2.351539,-1.174149,-0.000086,-1.174149,-0.000086,1997
…,…,…,…,…,…,…
3.676806,1.713317,4.279493,-1.0,4.279493,-1.0,8073
3.700655,1.737165,4.284375,-1.0,4.284375,-1.0,8424
3.670718,1.707221,4.287497,-1.0,4.287497,-1.0,8184


In [92]:
f_col = "f_1_min"
c_col = "c_1_min"

front = df_front.select([f_col, c_col]).drop_nulls().filter(pl.col(f_col).is_finite() & pl.col(c_col).is_finite())

f_ideal, f_nadir = front.select(
    pl.min(f_col).alias("f_ideal"),
    pl.max(f_col).alias("f_nadir"),
)

front_norm = front.with_columns(
    ((pl.col(f_col) - f_ideal) / (f_nadir - f_ideal)).alias("f_norm"),
)

norm_optimal_value = (optimal_value - f_ideal) / (f_nadir - f_ideal)
norm_optimal_value

f_ideal
f64
0.039118


In [93]:
low_th_norm = norm_optimal_value - 0.01 * norm_optimal_value
med_th_norm = norm_optimal_value - 0.05 * norm_optimal_value
hig_th_norm = norm_optimal_value - 0.10 * norm_optimal_value

print(med_th_norm)

deltas = front_norm.with_columns(
    (abs(pl.col("f_norm") - low_th_norm)).alias("low_delta"),
    (abs(pl.col("f_norm") - med_th_norm)).alias("med_delta"),
    (abs(pl.col("f_norm") - hig_th_norm)).alias("hig_delta"),
)

low_ct = deltas.sort("low_delta").select(pl.first(c_col))
med_ct = deltas.sort("med_delta").select(pl.first(c_col))
hig_ct = deltas.sort("hig_delta").select(pl.first(c_col))

print(f"{low_ct=}\n, {med_ct=}\n, {hig_ct=}\n")


shape: (1,)
Series: 'f_ideal' [f64]
[
	0.037162
]
low_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 0.002311 │
└──────────┘
, med_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 0.011739 │
└──────────┘
, hig_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 0.023762 │
└──────────┘

